In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# Check performance nn.Conv2d

In [2]:
#export
from exp.nb_02 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Signature: torch.nn.modules.conv._ConvNd.reset_parameters(self)
Docstring: <no docstring>
Source:   
    def reset_parameters(self):
        n = self.in_channels
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            init.uniform_(self.bias, -bound, bound)
File:      ~/anaconda3/lib/python3.7/site-packages/torch/nn/modules/conv.py
Type:      function


In [4]:
# Quick train something
x_train,y_train,x_valid,y_valid = get_data()
train_mean,train_std = x_train.mean(),x_train.std()
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [12]:
x_train.shape, y_train.max()

(torch.Size([50000, 1, 28, 28]), tensor(9))

In [6]:
x_train = x_train.view(-1, 1, 28, 28)
x_valid = x_valid.view(-1, 1, 28, 28)
x_train.shape, x_valid.shape

(torch.Size([50000, 1, 28, 28]), torch.Size([10000, 1, 28, 28]))

In [14]:
n, *_ = x_train.shape
c = y_train.max()+1
nh = 32
n, c

(50000, tensor(10))

In [65]:
l1 = nn.Conv2d(1, nh, 5) # input, output filter, 5x5 kernel

In [16]:
x = x_valid[:100]

In [66]:
x.shape

torch.Size([100, 1, 28, 28])

In [24]:
def stats(x): return x.mean(), x.std()

In [21]:
l1.weight.shape    # 1channelx5x5 size kernel, with 32 kernels 

torch.Size([32, 1, 5, 5])

In [25]:
stats(l1.weight), stats(l1.bias)

((tensor(-0.0009, grad_fn=<MeanBackward1>),
  tensor(0.1161, grad_fn=<StdBackward0>)),
 (tensor(0.0195, grad_fn=<MeanBackward1>),
  tensor(0.1217, grad_fn=<StdBackward0>)))

In [26]:
t = l1(x)

In [28]:
t.shape, stats(t)

(torch.Size([100, 32, 24, 24]),
 (tensor(0.0170, grad_fn=<MeanBackward1>),
  tensor(0.6719, grad_fn=<StdBackward0>)))

## Kaiming normal

In [52]:
init.kaiming_normal_(l1.weight, a=1.)
stats(l1.weight), stats(l1(x))

((tensor(-0.0093, grad_fn=<MeanBackward1>),
  tensor(0.1984, grad_fn=<StdBackward0>)),
 (tensor(-0.0031, grad_fn=<MeanBackward1>),
  tensor(1.0887, grad_fn=<StdBackward0>)))

In [32]:
import torch.nn.functional as F

In [33]:
def f1(x, a=0): return F.leaky_relu(l1(x), a)

In [70]:
init.kaiming_normal_(l1.weight, a=0)
stats(f1(x))

(tensor(0.5304, grad_fn=<MeanBackward1>),
 tensor(0.9416, grad_fn=<StdBackward0>))

In [71]:
l1 = nn.Conv2d(1, nh, 5)    #? Hmmm Without init? 
stats(f1(x))

(tensor(0.2332, grad_fn=<MeanBackward1>),
 tensor(0.4003, grad_fn=<StdBackward0>))

In [72]:
# receptive field size ????
rec_fs = l1.weight[0, 0].numel()
rec_fs

25

In [74]:
nf, ni, *_ = l1.weight.shape
nf, ni

(32, 1)

In [75]:
fan_in  = ni*rec_fs
fan_out = nf*rec_fs
fan_in,fan_out

(25, 800)

In [76]:
def gain(a): return math.sqrt(2.0 / (1 + a**2))

In [77]:
gain(1),gain(0),gain(0.01),gain(0.1),gain(math.sqrt(5.))   # sqrt(5) is not good

(1.0,
 1.4142135623730951,
 1.4141428569978354,
 1.4071950894605838,
 0.5773502691896257)

In [78]:
torch.zeros(10000).uniform_(-1,1).std()  # get uniform

tensor(0.5735)

In [79]:
1/math.sqrt(3)

0.5773502691896258

In [80]:
def kaiming2(x,a, use_fan_out=False):
    nf,ni,*_ = x.shape
    rec_fs = x[0,0].shape.numel()
    fan = nf*rec_fs if use_fan_out else ni*rec_fs
    std = gain(a) / math.sqrt(fan)
    bound = math.sqrt(3.) * std
    x.data.uniform_(-bound,bound)

In [81]:
kaiming2(l1.weight, a=0)
stats(f1(x))

(tensor(0.5310, grad_fn=<MeanBackward1>),
 tensor(0.9479, grad_fn=<StdBackward0>))

In [82]:
kaiming2(l1.weight, a=math.sqrt(5.))
stats(f1(x))   # problem

(tensor(0.2128, grad_fn=<MeanBackward1>),
 tensor(0.3840, grad_fn=<StdBackward0>))

In [83]:
class Flatten(nn.Module):
    def forward(self,x): return x.view(-1)

### Normal NN

In [84]:
m = nn.Sequential(
    nn.Conv2d(1,8, 5,stride=2,padding=2), nn.ReLU(),
    nn.Conv2d(8,16,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(16,32,3,stride=2,padding=1), nn.ReLU(),
    nn.Conv2d(32,1,3,stride=2,padding=1),
    nn.AdaptiveAvgPool2d(1),
    Flatten(),
)

In [85]:
y = y_valid[:100].float()

In [86]:
t = m(x)
stats(t)

(tensor(0.0277, grad_fn=<MeanBackward1>),
 tensor(0.0057, grad_fn=<StdBackward0>))

In [87]:
l = mse(t,y)
l.backward()

In [88]:
stats(m[0].weight.grad)

(tensor(0.0003), tensor(0.0164))

In [89]:
init.kaiming_uniform_??

Signature:
init.kaiming_uniform_(
    ['tensor', 'a=0', "mode='fan_in'", "nonlinearity='leaky_relu'"],
)
Source:   
def kaiming_uniform_(tensor, a=0, mode='fan_in', nonlinearity='leaky_relu'):
    r"""Fills the input `Tensor` with values according to the method
    described in "Delving deep into rectifiers: Surpassing human-level
    performance on ImageNet classification" - He, K. et al. (2015), using a
    uniform distribution. The resulting tensor will have values sampled from
    :math:`\mathcal{U}(-\text{bound}, \text{bound})` where

    .. math::
        \text{bound} = \sqrt{\frac{6}{(1 + a^2) \times \text{fan\_in}}}

    Also known as He initialization.

    Args:
        tensor: an n-dimensional `torch.Tensor`
        a: the negative slope of the rectifier used after this layer (0 for ReLU
            by default)
        mode: either 'fan_in' (default) or 'fan_out'. Choosing `fan_in`
            preserves the magnitude of the variance of the weights in the
            forward 

In [96]:
for l in m:
    if isinstance(l,nn.Conv2d):
        init.kaiming_uniform_(l.weight)
        l.bias.data.zero_()

In [97]:
t = m(x)
stats(t)

(tensor(-0.4873, grad_fn=<MeanBackward1>),
 tensor(0.3674, grad_fn=<StdBackward0>))

In [98]:
l = mse(t,y)
l.backward()
stats(m[0].weight.grad)  # ??? This is very good

(tensor(-0.1790), tensor(0.8740))